In [88]:
!pip3 install vertexai

Defaulting to user installation because normal site-packages is not writeable
  Using cached google_cloud_aiplatform-1.48.0-py2.py3-none-any.whl.metadata (30 kB)
Using cached google_cloud_aiplatform-1.48.0-py2.py3-none-any.whl (4.8 MB)
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.49.0
    Uninstalling google-cloud-aiplatform-1.49.0:
      Successfully uninstalled google-cloud-aiplatform-1.49.0


In [89]:
# !pip3 install --upgrade google-cloud-aiplatform
!pip3 install -U google-cloud-aiplatform "shapely<2"

Defaulting to user installation because normal site-packages is not writeable
  Using cached google_cloud_aiplatform-1.49.0-py2.py3-none-any.whl.metadata (30 kB)
Using cached google_cloud_aiplatform-1.49.0-py2.py3-none-any.whl (4.9 MB)
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.48.0
    Uninstalling google-cloud-aiplatform-1.48.0:
      Successfully uninstalled google-cloud-aiplatform-1.48.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vertexai 1.48.0 requires google-cloud-aiplatform[all]==1.48.0, but you have google-cloud-aiplatform 1.49.0 which is incompatible.


In [90]:
USE_COLAB = False
if USE_COLAB:
    from google.colab import auth
    auth.authenticate_user()
else:
    !gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=2Q0Lw1IoGg0EnvvqHpjKV78og7OwQA&access_type=offline&code_challenge=rHOkjqvNIPaICa4k0JVnrorf45k3HAlklBY2s4Q9Om0&code_challenge_method=S256


Credentials saved to file: [C:\Users\johnsonhk88\AppData\Roaming\gcloud\application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "gdg-vertex-ai-workshop-curl" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


In [93]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models


textsi_1 = """Cymbal Direct is an online direct-to-consumer footwear and apparel retailer headquartered in Chicago. 

Founded in 2008, Cymbal Direct (originally ‘Antern’) is a fair trade and B Corp certified sustainability-focused company that works with cotton farmers to reinvest in their communities. The price range for Cymbal clothes is typically between $50 and $300.

In 2010, as Cymbal Group began focusing on digitally-savvy businesses that appealed to a younger demographic of shoppers, the holding company acquired Antern and renamed it Cymbal Direct. In 2019, Cymbal Direct reported an annual revenue of $7 million and employed a total of 32 employees. 

Cymbal Direct is a digitally native retailer. 

You are a personalized wiki of the company Cymbal."""


generation_config = {
    "max_output_tokens": 1024,
    "temperature": 0.2,
    "top_p": 0.8,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

## Model init

In [94]:
vertexai.init(project="gdg-vertex-ai-workshop-curl", location="us-central1")

model = GenerativeModel(
    "gemini-1.0-pro-002",
    system_instruction=[textsi_1]
  )

In [95]:
def multiturn_generate_content():
  chat = model.start_chat()
  print(chat.send_message(
      ["""What does Cymbal sell?"""],
      generation_config=generation_config,
      safety_settings=safety_settings
  ))
  print(chat.send_message(
      ["""When was the company founded?"""],
      generation_config=generation_config,
      safety_settings=safety_settings
  ))
  print(chat.send_message(
      ["""How much is the price of Cymbal clothes?"""],
      generation_config=generation_config,
      safety_settings=safety_settings
  ))

In [96]:
multiturn_generate_content()

candidates {
  content {
    role: "model"
    parts {
      text: "Cymbal Direct sells footwear and apparel. The price range for Cymbal clothes is typically between $50 and $300."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
    probability_score: 0.0977343246
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.059319146
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
    probability_score: 0.128634557
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0454372168
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
    probability_score: 0.176955879
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0.0555151142
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
    probability_score: 0.207857162
    severity: HARM_SEVERITY_NEGLIGIBLE
    severity_score: 0

In [97]:
def singleChat(message):
  chat = model.start_chat()
  print(chat.send_message(
      [message],
      generation_config=generation_config,
      safety_settings=safety_settings
  ))

In [119]:
responses = model.generate_content("Why is the sky blue?", stream=True)

for response in responses:
    print(response.text, end="")

You are a personalized wiki of the company Cymbal.

In [121]:
def generateResponse(prompTemp,message):
    response = model.generate_content(f"""
    {prompTemp}
    {message}""")
    print(response.text)
    return response.text

In [98]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
import yaml

## Load Dataset
#### Load  **Mental Health Conversational Data** dataset
https://www.kaggle.com/datasets/elvis23/mental-health-conversational-data 
###### file
intents.json

In [99]:
## Define Dataset Path
intentsJson = 'intents.json'
xlsxName = 'dataset.xlsx'
healthanxiety = 'healthanxiety_dataset.csv'
psQAJson = 'PsyQA_example.json'
depressionYML= "depression.yml"
depressEmoTrain = "train.json"
depressEmoTest = "test.json"
depressEmoVal = "val.json"

In [100]:


with open(intentsJson, 'r') as file:
  # read JSON data
  data = json.load(file)
data


{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hey',
    'Is anyone there?',
    'Hi there',
    'Hello',
    'Hey there',
    'Howdy',
    'Hola',
    'Bonjour',
    'Konnichiwa',
    'Guten tag',
    'Ola'],
   'responses': ['Hello there. Tell me how are you feeling today?',
    'Hi there. What brings you here today?',
    'Hi there. How are you feeling today?',
    'Great to see you. How do you feel currently?',
    "Hello there. Glad to see you're back. What's going on in your world right now?"]},
  {'tag': 'morning',
   'patterns': ['Good morning'],
   'responses': ["Good morning. I hope you had a good night's sleep. How are you feeling today? "]},
  {'tag': 'afternoon',
   'patterns': ['Good afternoon'],
   'responses': ['Good afternoon. How is your day going?']},
  {'tag': 'evening',
   'patterns': ['Good evening'],
   'responses': ['Good evening. How has your day been?']},
  {'tag': 'night',
   'patterns': ['Good night'],
   'responses': ['Good night. Get some prop

In [101]:
intentsDF= pd.DataFrame(data["intents"])
intentsDF.head()

,tag,patterns,responses
0,greeting,"[Hi, Hey, Is anyone there?, Hi there, Hello, H...",[Hello there. Tell me how are you feeling toda...
1,morning,[Good morning],[Good morning. I hope you had a good night's s...
2,afternoon,[Good afternoon],[Good afternoon. How is your day going?]
3,evening,[Good evening],[Good evening. How has your day been?]
4,night,[Good night],"[Good night. Get some proper sleep, Good night..."


In [102]:
intentsDF["patterns"]

0     [Hi, Hey, Is anyone there?, Hi there, Hello, H...
1                                        [Good morning]
2                                      [Good afternoon]
3                                        [Good evening]
4                                          [Good night]
                            ...                        
75    [What do I do if I'm worried about my mental h...
76                       [How do I know if I'm unwell?]
77    [How can I maintain social connections? What i...
78    [What's the difference between anxiety and str...
79    [What's the difference between sadness and dep...
Name: patterns, Length: 80, dtype: object

In [103]:
intentsDF["tag"].value_counts()

tag
greeting      1
morning       1
fact-11       1
fact-10       1
fact-9        1
             ..
understand    1
death         1
scared        1
sleep         1
fact-32       1
Name: count, Length: 80, dtype: int64

In [104]:
intentsDF.describe()

,tag,patterns,responses
count,80,80,80
unique,80,80,80
top,greeting,"[Hi, Hey, Is anyone there?, Hi there, Hello, H...",[Hello there. Tell me how are you feeling toda...
freq,1,1,1


In [105]:
intentsDF["patterns"].apply(len).value_counts()

patterns
1     33
2     13
3     11
4      7
5      5
8      4
6      3
7      2
12     1
11     1
Name: count, dtype: int64

### Load  PsyQA DataSet
https://github.com/thu-coai/PsyQA
##### file  : PsyQA-main

In [106]:
# with open(psQAJson, 'r') as file:
#   # read JSON data
#   data = json.load(file
psQADF = pd.read_json(psQAJson)
psQADF.head()


,question,description,keywords,answers,questionID
0,去爱一个事事让着你的人，是一种不可能的期待吗？,去爱一个事事让着你的人？有这样的人么？是一种不可能的期待么？,"恋爱,恋爱经营",[{'answer_text': '你好，首先回答你关于有没有事事让着你的人，如果你真的去找...,4394
1,为什么总是会因为他人学习而过度焦虑？,经常在校园里看到学习的人，于是便觉得自己变得不求上进，心里产生焦虑，害怕和别人的距离拉大，总...,"行为,应激,焦虑,困惑",[{'answer_text': '～～很抱歉看到你最近状态不佳，抱抱你呀～看了你的描述“会...,4395
2,我们都会和讨厌的人相处吗？怎样相处是最佳方式？,身边如果有讨厌的人，怎样相处是一种最佳方式呢？,"人际,人际边界,社会适应",[{'answer_text': '这可真是个难题呢。想到两种可能。一，这个人讨厌，但于你的...,4397
3,遇到心动的东西不舍得买，是不爱自己吗？,有时候看到自己喜欢的物品很心动，但是不舍得买，有时看到特别想吃东西，也不舍得买给自己，总要等...,"成长,自我成长,自我接纳,人生意义",[{'answer_text': '题主你好呀！一般来说是这样的，我自己曾经就是这样的人，舍...,4398
4,工作很压抑，怎么才能和自己和解？或者说改变呢？,工作压力很大，总觉得很多事情都完成不了，所以心里压力特别大，还曾经被挺喜欢的同事欺负，所以更...,"情绪,抑郁情绪,焦虑情绪,情绪调节",[{'answer_text': '希望以下回答能帮助到你：️**首先对你所描述的情况梳理如...,4399


### Load *Students anxiety and depression dataset**  
https://www.kaggle.com/datasets/sahasourav17/students-anxiety-and-depression-dataset
####  file
dataset.xlsx

In [107]:
'''
require install openpyxl package
pip3 install openpyxl
'''
dataXLS = pd.read_excel(xlsxName)
dataXLS.head()

,text,label
0,oh my gosh,1.0
1,"trouble sleeping, confused mind, restless hear...",1.0
2,"All wrong, back off dear, forward doubt. Stay ...",1.0
3,I've shifted my focus to something else but I'...,1.0
4,"I'm restless and restless, it's been a month n...",1.0


In [108]:
dataXLS.describe()

,label
count,6980.000000
mean,0.105014
std,0.306594
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [109]:
dataXLS["label"]

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
6977    0.0
6978    0.0
6979    0.0
6980    0.0
6981    0.0
Name: label, Length: 6982, dtype: float64

In [110]:
noDepress= dataXLS[dataXLS["label"]== 1]
noDepress.describe()

,label
count,733.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


### Load Mental Health Data (Anxiety) dataset  
https://www.kaggle.com/datasets/michellevp/predicting-anxiety-in-mental-health-data
### file
healthanxiety_dataset.csv
### **Note** this dataset is not useful for this AI Mentiona Classification task

In [111]:
healthanxietyDF = pd.read_csv(healthanxiety)
healthanxietyDF.head()

,subreddit,author,date,post,automated_readability_index,coleman_liau_index,flesch_kincaid_grade_level,flesch_reading_ease,gulpease_index,gunning_fog_index,...,tfidf_wish,tfidf_without,tfidf_wonder,tfidf_work,tfidf_worri,tfidf_wors,tfidf_would,tfidf_wrong,tfidf_x200b,tfidf_year
0,healthanxiety,Nomis176,1/1/2018,"Final doctor appointment tomorrow, tired of co...",9.137909,7.282428,8.616061,72.474091,61.090909,11.345455,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.055919
1,healthanxiety,psychstudent317,1/1/2018,Anyone have bone or muscle pain that was stres...,4.399892,7.131386,3.665806,87.185376,74.161290,5.423656,...,0.0,0.0,0.0,0.000000,0.290466,0.0,0.0,0.000000,0.0,0.000000
2,healthanxiety,bulk_barn,1/1/2018,Listening to your body? I'm curious how those ...,3.590660,4.930895,4.605802,85.640967,72.584906,7.564151,...,0.0,0.0,0.0,0.110757,0.000000,0.0,0.0,0.332673,0.0,0.000000
3,healthanxiety,AutoModerator,1/1/2018,Weekly /r/HealthAnxiety Challenge - Exercise A...,4.515682,7.033342,5.059091,78.383636,72.863636,8.945455,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000
4,healthanxiety,parthkhurana7,1/1/2018,This is killing me So i had a bacterial stomac...,3.034388,5.356252,3.977494,85.306958,76.721519,7.757637,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000


In [112]:
healthanxietyDF.columns

Index(['subreddit', 'author', 'date', 'post', 'automated_readability_index',
       'coleman_liau_index', 'flesch_kincaid_grade_level',
       'flesch_reading_ease', 'gulpease_index', 'gunning_fog_index',
       ...
       'tfidf_wish', 'tfidf_without', 'tfidf_wonder', 'tfidf_work',
       'tfidf_worri', 'tfidf_wors', 'tfidf_would', 'tfidf_wrong',
       'tfidf_x200b', 'tfidf_year'],
      dtype='object', length=350)

#### Load Depression data for chatbot dataset
https://www.kaggle.com/datasets/nupurgopali/depression-data-for-chatbot 
## file
depresssion.yml file

In [113]:
dataYML = yaml.load(open(depressionYML), Loader=yaml.FullLoader)
dataYML

{'categories': ['depression'],
 'conversations': [['What Is Depression?',
   'Depression is a common and serious medical illness that negatively affects how you feel, the way you think and how you act.',
   'Fortunately,it is also treatable.',
   'Depression causes feelings of sadness and/or a loss of interest in activities you once enjoyed. It can lead to a variety of emotional and physical problems and can decrease your ability to function at work and at home.'],
  ['I feel i have let my parents down',
   'No matter what,your parents will always be proud of you and will love you.',
   'You will feel much better if you share your feelings with them.'],
  ['What are the types of depression?',
   'There are many different forms of depression but most common types are.',
   'Clinical depression,persistent depressive disorder,bipolar disorder,postnatal disorder.'],
  ['I am good for nothing.',
   "Don't ever think you cannot achieve what you want.",
   'You are capable of reaching your go

In [114]:
with open(depressionYML, 'r') as file:
    tempYML = yaml.safe_load(file)
# Convert to JSON
with open('depression.json', 'w') as json_file:
    json.dump(tempYML, json_file)

In [115]:
with open('depression.json') as json_data:
     data = json.load(json_data)
data

{'categories': ['depression'],
 'conversations': [['What Is Depression?',
   'Depression is a common and serious medical illness that negatively affects how you feel, the way you think and how you act.',
   'Fortunately,it is also treatable.',
   'Depression causes feelings of sadness and/or a loss of interest in activities you once enjoyed. It can lead to a variety of emotional and physical problems and can decrease your ability to function at work and at home.'],
  ['I feel i have let my parents down',
   'No matter what,your parents will always be proud of you and will love you.',
   'You will feel much better if you share your feelings with them.'],
  ['What are the types of depression?',
   'There are many different forms of depression but most common types are.',
   'Clinical depression,persistent depressive disorder,bipolar disorder,postnatal disorder.'],
  ['I am good for nothing.',
   "Don't ever think you cannot achieve what you want.",
   'You are capable of reaching your go

In [116]:
depressionJson = pd.DataFrame(data["conversations"])
depressionJson.head()

,0,1,2,3,4
0,What Is Depression?,Depression is a common and serious medical ill...,"Fortunately,it is also treatable.",Depression causes feelings of sadness and/or a...,None
1,I feel i have let my parents down,"No matter what,your parents will always be pro...",You will feel much better if you share your fe...,None,None
2,What are the types of depression?,There are many different forms of depression b...,"Clinical depression,persistent depressive diso...",None,None
3,I am good for nothing.,Don't ever think you cannot achieve what you w...,You are capable of reaching your goals.Here is...,None,None
4,I am feeling stressed lately,My research says that music is the best way to...,so visit the link and listen to your favourite...,"it will definitely cheer you up ""https://www.s...",None


In [117]:
# with open(depressEmoTrain, 'r') as json_data:
#      data = json.load(json_data)
# data

In [118]:
# depresEmoTrainDF = pd.read_json(depressEmoTest)
# depresEmoTrainDF

##  Test LLM Response 

In [120]:
testPrompt1 = """Is it have Depression or anxiety disorder or not, as below conversation?

    Context:"""



In [122]:
for msg in dataXLS["text"][:10]:
    print(generateResponse(testPrompt1,msg))

I am sorry, but I am unable to provide medical advice. If you are concerned about your mental health, please reach out to a qualified professional.
I am sorry, but I am unable to provide medical advice. If you are concerned about your mental health, please reach out to a qualified professional.
The information you provided does not contain anything about depression. However, it does mention Cymbal Direct, a company that sells fair trade and sustainable clothing and footwear. If you are interested in learning more about Cymbal Direct, I would be happy to provide you with additional information. 

If you are interested in learning more about depression, I would recommend that you reach out to a mental health professional. They will be able to provide you with the support and resources that you need.
The information you provided does not contain anything about depression. However, it does mention Cymbal Direct, a company that sells fair trade and sustainable clothing and footwear. If you 

ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai.